In [ ]:
import os, pandas as pd, networkx as nx, numpy as np, statsmodels.api as sm
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## load data

In [ ]:
# load stadium meta data
stadname_data = pd.read_csv('output/00_stadname_data.csv')
aid2name = {aid: name for aid, name in zip(stadname_data.area_id, stadname_data.area_name)}

aid2city = {aid: city for aid, city in sorted({
    1: 'München',
    2: 'Berlin',
    3: 'Leipzig',
    4: 'Frankfurt am Main',
    10: 'Dortmund',
    11: 'Stuttgart',
    15: 'Köln',
    24: 'Düsseldorf',
    25: 'Hamburg',
    28: 'Gelsenkirchen',

    16: 'Berlin',
    29: 'Hamburg',
    41: 'München',
    64: 'Frankfurt am Main',
}.items(), key=lambda item: item[1])}
city2aid = {city: aid for aid, city in [(aid, city) for aid, city in aid2city.items()][::-1]}

In [ ]:
# n_c Germany & stadiums: load total contact numbers for Germany and stadiums
data_germany = pd.read_csv(f'output/00_ncontacts_germany_{cdef}.csv')
data_germany['day'] = [d.date() for d in pd.to_datetime(data_germany.day)]

# n_c cities: load total contact numbers numbers for cities
data = pd.read_csv(f'output/00_ncontacts_cities_{cdef}.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]

# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]

In [ ]:
def convert_labels(match):
    if 'BLiga' in match:
        new = 'BLiga'
    elif 'S25' in match or 'friendly' in match or 'Am.' in match or 'HipHop' in match:
        new = ''
    else:
        new = match
    return new
match_data['match'] = match_data.match.apply(convert_labels)

In [ ]:
vac_data = [
    ['Berlin', date(2024,7,18), date(2024,8,30)],
    ['Dortmund', date(2024,7,8), date(2024,8,20)],
    ['Düsseldorf', date(2024,7,8), date(2024,8,20)],
    ['Frankfurt am Main', date(2024,7,15), date(2024,8,23)],
    ['Gelsenkirchen', date(2024,7,8), date(2024,8,20)],
    ['Hamburg', date(2024,7,18), date(2024,8,28)],
    ['Köln', date(2024,7,8), date(2024,8,20)],
    ['Leipzig', date(2024,6,20), date(2024,8,2)],
    ['München', date(2024,7,29), date(2024,9,9)],
    ['Stuttgart', date(2024,7,25), date(2024,9,7)],
]
vac_data = pd.DataFrame(vac_data, columns=['city','day_start','day_end'])
vac_data

## plot $C(t)$ ...

### ... by city, weekday-specific baseline/detrended

In [ ]:
#data_here = data.copy(deep=True)
data_1 = data.drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_1.reset_index()
data_2 = data[data.area_id>-.5].drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_2.reset_index()
data_here = data_1.merge(data_2, on=['day','city'], how='left')
data_here['ncontacts'] = data_here.ncontacts_1 + data_here.ncontacts_2.fillna(0.)
data_here['wd'] = [d.weekday() for d in data_here.day]

# baseline = number contacts in city without stadiums by weekday, averaged over many weeks
baseline = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.mean()).rename(columns={'ncontacts_1':'baseline'}).reset_index()
data_here = data_here.merge(baseline, on=['city','wd'])

data_here2, data_here3 = data_here.copy(deep=True), data_here.copy(deep=True)
data_here2['to_baseline'] = data_here.ncontacts_1/data_here.baseline
data_here2['with_stadiums'] = False
data_here3['to_baseline'] = (data_here.ncontacts_1+data_here.ncontacts_2.fillna(0.))/data_here.baseline
data_here3['with_stadiums'] = True
data_here4 = pd.concat([data_here2, data_here3], ignore_index=True)

#baseline_ci = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.std()).rename(columns={'ncontacts_1':'baseline_std'}).reset_index()
#baseline_ci['lower'] = 1. - baseline_ci.baseline_std / baseline.baseline
#baseline_ci['upper'] = 1. + baseline_ci.baseline_std / baseline.baseline

data_here4

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

city_list = sorted(set(list(aid2city.values()))) #set(data_here.city)

# Plot the lines on two facets
g = sns.relplot(
    data=data_here4,
    x="day", y="to_baseline",
    hue="with_stadiums", hue_order=[True,False],
    row="city", row_order=city_list,# size="choice",# col="align",
    kind="line", palette=palette,# size_order=[False, True], 
    height=2, aspect=5.5, facet_kws=dict(sharex=False),
)

ylimup = 3.75
axes = g.axes
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax_row, city in zip(axes, city_list):
    for ax in ax_row:
        ax.set_ylabel('contacts')
        ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data_here.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data_here.day))])#, rotation=90)
        #lower = set(data_here.city)
        #ax.fill_between([data_here.day.min(), data_here.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data_here4.day.min() and day <= data.day.max():
                if data_here4[(data_here4.day==day) & (data_here4.city==city)].to_baseline.max() < 2.5:
                    ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
                else:
                    ax.text(day, 0.1, match, rotation=90, ha='center', va='bottom', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [0]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([0., ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)

lg = g._legend
lg.set_title('stadiums')
for tx in lg.texts:
    if tx.get_text() == 'True':
        tx.set_text('with')
    elif tx.get_text() == 'False':
        tx.set_text('without')

plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp.pdf', bbox_inches='tight')
plt.show()

In [ ]:
cols = ['day','city','with_stadiums','to_baseline','ncontacts_1','ncontacts_2']
tmp = data_here4[(data_here4.day>=di) & (data_here4.day<=df)][cols]
left = tmp[tmp.with_stadiums==False][[c for c in cols if c not in ['ncontacts_2', 'with_stadiums']]]
right = tmp[tmp.with_stadiums==True][[c for c in cols if c not in ['ncontacts_1', 'with_stadiums']]]
for_andrzej = left.merge(right, on=['day','city'], suffixes=('_1','_2'))
for_andrzej = for_andrzej[['day','city','ncontacts_1','ncontacts_2','to_baseline_1','to_baseline_2']]
reg2reg = {'ncontacts_1':'contacts_city', 'ncontacts_2':'contacts_stad',
           'to_baseline_1':'contact_to_baseline_city', 'to_baseline_2':'contact_to_baseline_all'}
for_andrzej = for_andrzej.rename(columns=reg2reg).sort_values(['day','city']).reset_index(drop=True)
for_andrzej['contacts_stad'] = for_andrzej.contacts_stad.fillna(0).astype(int)
for_andrzej.head(20)

In [ ]:
for_andrzej.to_csv(f'output/03_contactlevel_daily_{cdef_alt}_bycity_unscaled_detrended.csv', index=False, encoding='utf-8-sig')

### ... by city, global baseline

In [ ]:
#data_here = data.copy(deep=True)
data_1 = data.drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_1.reset_index()
data_2 = data[data.area_id>-.5].drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_2.reset_index()
data_here = data_1.merge(data_2, on=['day','city'], how='left')
data_here['ncontacts'] = data_here.ncontacts_1 + data_here.ncontacts_2.fillna(0.)
data_here['wd'] = [d.weekday() for d in data_here.day]

# baseline = number contacts in city without stadiums by weekday, averaged over many weeks
baseline = pd.DataFrame(data_here.groupby(['city']).ncontacts_1.mean()).rename(columns={'ncontacts_1':'baseline'}).reset_index()
data_here = data_here.merge(baseline, on=['city'])

data_here2, data_here3 = data_here.copy(deep=True), data_here.copy(deep=True)
data_here2['to_baseline'] = data_here.ncontacts_1/data_here.baseline
data_here2['with_stadiums'] = False
data_here3['to_baseline'] = (data_here.ncontacts_1+data_here.ncontacts_2.fillna(0.))/data_here.baseline
data_here3['with_stadiums'] = True
data_here4 = pd.concat([data_here2, data_here3], ignore_index=True)

data_here4

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

city_list = sorted(set(list(aid2city.values()))) #set(data_here.city)

# Plot the lines on two facets
g = sns.relplot(
    data=data_here4,
    x="day", y="to_baseline",
    hue="with_stadiums", hue_order=[True,False],
    row="city", row_order=city_list,# size="choice",# col="align",
    kind="line", palette=palette,# size_order=[False, True], 
    height=2, aspect=5.5, facet_kws=dict(sharex=False),
)

ylimup = 4.25
axes = g.axes
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax_row, city in zip(axes, city_list):
    for ax in ax_row:
        ax.set_ylabel('contacts')
        ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data_here.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data_here.day))])#, rotation=90)
        #lower = set(data_here.city)
        #ax.fill_between([data_here.day.min(), data_here.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data_here4.day.min() and day <= data.day.max():
                if data_here4[(data_here4.day==day) & (data_here4.city==city)].to_baseline.max() < 2.5:
                    ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
                else:
                    ax.text(day, 0.1, match, rotation=90, ha='center', va='bottom', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [0]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([0., ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)

lg = g._legend
lg.set_title('stadiums')
for tx in lg.texts:
    if tx.get_text() == 'True':
        tx.set_text('with')
    elif tx.get_text() == 'False':
        tx.set_text('without')

plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_wd.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_wd.pdf', bbox_inches='tight')
plt.show()

### ... Germany, weekday-specific baseline/detrended

In [ ]:
#data_here = data.copy(deep=True)
#data_1 = data.drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_1.reset_index()
#data_2 = data[data.area_id>-.5].drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_2.reset_index()
data_here = data_germany[['day','ncontacts_1','ncontacts_2']].copy(deep=True)#data_1.merge(data_2, on=['day','city'], how='left')
data_here['ncontacts'] = data_here.ncontacts_1 + data_here.ncontacts_2.fillna(0.)
data_here['wd'] = [d.weekday() for d in data_here.day]

# baseline = number contacts in city without stadiums by weekday, averaged over many weeks
baseline = pd.DataFrame(data_here.groupby(['wd']).ncontacts_1.mean()).rename(columns={'ncontacts_1':'baseline'}).reset_index()
data_here = data_here.merge(baseline, on=['wd'])

data_here2, data_here3 = data_here.copy(deep=True), data_here.copy(deep=True)
data_here2['to_baseline'] = data_here.ncontacts_1/data_here.baseline
data_here2['with_stadiums'] = False
data_here3['to_baseline'] = (data_here.ncontacts_1+data_here.ncontacts_2.fillna(0.))/data_here.baseline
data_here3['with_stadiums'] = True
data_here4 = pd.concat([data_here2, data_here3], ignore_index=True)

#baseline_ci = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.std()).rename(columns={'ncontacts_1':'baseline_std'}).reset_index()
#baseline_ci['lower'] = 1. - baseline_ci.baseline_std / baseline.baseline
#baseline_ci['upper'] = 1. + baseline_ci.baseline_std / baseline.baseline

data_here4

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

city_list = sorted(set(list(aid2city.values()))) #set(data_here.city)

# Plot the lines on two facets
g = sns.relplot(
    data=data_here4,
    x="day", y="to_baseline",
    hue="with_stadiums", hue_order=[True,False],
    kind="line", palette=palette,# size_order=[False, True], 
    height=2, aspect=5.5, facet_kws=dict(sharex=False),
)

axes = g.axes
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax_row, city in zip(axes, ['Germany']):
    for ax in ax_row:
        ax.set_ylabel('contacts')
        ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data_here.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data_here.day))])#, rotation=90)
        #lower = set(data_here.city)
        #ax.fill_between([data_here.day.min(), data_here.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data_here4.day.min() and day <= data.day.max():
                ax.text(day, 3., match, rotation=90, ha='center', va='top')
        #day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        #day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        #if day_end > day_start:
        #    ax.fill_between([day_start, day_end], [0]*2, [3]*2, color='gray', alpha=.25)
        ax.set_ylim([.75, 1.5])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [3]*2, color='C3', alpha=.25)
        #if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
        holi = date(2024,5,30)
        ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [3]*2, color='C3', alpha=.25*.5)

for city in set(vac_data.city):
    day_start = vac_data[vac_data.city==city].day_start.iloc[0]
    day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
    if day_end > day_start:
        ax.fill_between([day_start, day_end], [0]*2, [ylimup]*2, color='gray', alpha=.1)

lg = g._legend
lg.set_title('stadiums')
for tx in lg.texts:
    if tx.get_text() == 'True':
        tx.set_text('with')
    elif tx.get_text() == 'False':
        tx.set_text('without')

plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_germany.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_germany.pdf', bbox_inches='tight')
plt.show()

In [ ]:
cols = ['day','with_stadiums','to_baseline']
reg2reg = {'to_baseline':'contactlevel_to_baseline_ratio'}
for_andrzej = data_here4[(data_here4.day>=di) & (data_here4.day<=df)][cols]\
    .sort_values(['day','with_stadiums'])\
    .rename(columns=reg2reg)\
    .reset_index(drop=True)

In [ ]:
cols = ['day','with_stadiums','to_baseline','ncontacts_1','ncontacts_2']
tmp = data_here4[(data_here4.day>=di) & (data_here4.day<=df)][cols]
left = tmp[tmp.with_stadiums==False][[c for c in cols if c not in ['ncontacts_2', 'with_stadiums']]]
right = tmp[tmp.with_stadiums==True][[c for c in cols if c not in ['ncontacts_1', 'with_stadiums']]]
for_andrzej = left.merge(right, on=['day'], suffixes=('_1','_2'))
for_andrzej = for_andrzej[['day','ncontacts_1','ncontacts_2','to_baseline_1','to_baseline_2']]
reg2reg = {'ncontacts_1':'contacts_germ', 'ncontacts_2':'contacts_stad',
           'to_baseline_1':'contact_to_baseline_germ', 'to_baseline_2':'contact_to_baseline_all'}
for_andrzej = for_andrzej.rename(columns=reg2reg).sort_values(['day']).reset_index(drop=True)
for_andrzej['contacts_stad'] = for_andrzej.contacts_stad.fillna(0).astype(int)
for_andrzej.head(20)

In [ ]:
for_andrzej.to_csv(f'output/03_contactlevel_daily_{cdef_alt}_germany_unscaled_detrended.csv', index=False, encoding='utf-8-sig')

### ... Germany, global baseline

In [ ]:
#data_here = data.copy(deep=True)
#data_1 = data.drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_1.reset_index()
#data_2 = data[data.area_id>-.5].drop(columns=['area_id']).groupby(['day','city']).sum().ncontacts_2.reset_index()
data_here = data_germany[['day','ncontacts_1','ncontacts_2']].copy(deep=True)#data_1.merge(data_2, on=['day','city'], how='left')
data_here['ncontacts'] = data_here.ncontacts_1 + data_here.ncontacts_2.fillna(0.)
data_here['wd'] = [d.weekday() for d in data_here.day]

# baseline = number contacts in city without stadiums by weekday, averaged over many weeks
data_here['baseline'] = data_here.ncontacts_1.mean()

data_here2, data_here3 = data_here.copy(deep=True), data_here.copy(deep=True)
data_here2['to_baseline'] = data_here.ncontacts_1/data_here.baseline
data_here2['with_stadiums'] = False
data_here3['to_baseline'] = (data_here.ncontacts_1+data_here.ncontacts_2.fillna(0.))/data_here.baseline
data_here3['with_stadiums'] = True
data_here4 = pd.concat([data_here2, data_here3], ignore_index=True)

#baseline_ci = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.std()).rename(columns={'ncontacts_1':'baseline_std'}).reset_index()
#baseline_ci['lower'] = 1. - baseline_ci.baseline_std / baseline.baseline
#baseline_ci['upper'] = 1. + baseline_ci.baseline_std / baseline.baseline

data_here4

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

city_list = sorted(set(list(aid2city.values()))) #set(data_here.city)

# Plot the lines on two facets
g = sns.relplot(
    data=data_here4,
    x="day", y="to_baseline",
    hue="with_stadiums", hue_order=[True,False],
    kind="line", palette=palette,# size_order=[False, True], 
    height=2, aspect=5.5, facet_kws=dict(sharex=False),
)

axes = g.axes
holis = [date(2024,5,1), date(2024,5,9), date(2024,5,20)]
for ax_row, city in zip(axes, ['Germany']):
    for ax in ax_row:
        ax.set_ylabel('contacts')
        ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data_here.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data_here.day))])#, rotation=90)
        #lower = set(data_here.city)
        #ax.fill_between([data_here.day.min(), data_here.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data_here4.day.min() and day <= data.day.max():
                ax.text(day, 3., match, rotation=90, ha='center', va='top')
        #day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        #day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        #if day_end > day_start:
        #    ax.fill_between([day_start, day_end], [0]*2, [3]*2, color='gray', alpha=.25)
        ax.set_ylim([.75, 1.5])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [3]*2, color='C3', alpha=.25)
        #if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
        holi = date(2024,5,30)
        ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [3]*2, color='C3', alpha=.25*.5)

for city in set(vac_data.city):
    day_start = vac_data[vac_data.city==city].day_start.iloc[0]
    day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
    if day_end > day_start:
        ax.fill_between([day_start, day_end], [0]*2, [ylimup]*2, color='gray', alpha=.1)

lg = g._legend
lg.set_title('stadiums')
for tx in lg.texts:
    if tx.get_text() == 'True':
        tx.set_text('with')
    elif tx.get_text() == 'False':
        tx.set_text('without')

plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_wd_germany.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/03_contacts_baseline_cmp_wd_germany.pdf', bbox_inches='tight')
plt.show()